#### 节点为 HH, 使用切割法实现螺旋波

In [195]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from numba import njit, prange
from import_fun import HH, Diffusion2D, show_spiral_wave, FlowVelocity

In [196]:
# 开启交互式画图
mpl.use('TkAgg') # TkAgg Qt5Agg
plt.ion()  # 开启交互模式

In [197]:
Nx = 100
Ny = 100
method = "euler"
dt = 0.01

In [198]:
# 生成节点，初始值设定
nodes = HH(N=Nx*Ny, method=method, dt=dt)
nodes.vars_nodes[0] = 0.
nodes.vars_nodes[1] = 0.
nodes.vars_nodes[2] = 0.
nodes.vars_nodes[3] = 0.
nodes.params_nodes["Iex"] = 0.
nodes.spiking = False    # 关掉峰值探测器

# 生成一个方阵视图，这个视图与原始数组共享内存
V_view = nodes.vars_nodes[0].reshape(Nx, Ny)
m_view = nodes.vars_nodes[1].reshape(Nx, Ny)
h_view = nodes.vars_nodes[2].reshape(Nx, Ny)
n_view = nodes.vars_nodes[3].reshape(Nx, Ny)

I_input = np.zeros(Nx*Ny)
I_input_view = I_input.reshape(Nx, Ny)


In [199]:
# 设定动态显示器
spiral_wave = show_spiral_wave(nodes.vars_nodes[0], Nx, Ny, save_gif=False)

In [200]:
# 设定扩散耦合
## "No_flow", "periodic"
syn = Diffusion2D(D=0.6, boundary="No_flow", adjacency=8)

In [201]:
# mem_list = []
for i in range(100_00):
    nodes()

nodes.t = 0.
# I_input_view[:, 0] = 10

mod = int(Nx/2)
for i in range(60_00):
    if i<10_00:
        I_input_view[:, 0] = 10
    else:
        I_input_view[:, 0] = 0.

    if i == 22_00:
        V_view[mod:, :] = -65.00095684
        m_view[mod:, :] = 0.05292585
        h_view[mod:, :] = 0.59450462
        n_view[mod:, :] = 0.31761895

    spiral_wave(i, nodes.t, show_interval=1_00)
    Isyn = syn(V_view)
    nodes(Isyn+I_input)

fvelocity = FlowVelocity(V_view.copy(), alpha=20.)

for i in range(60_00):
    if i%6_00 == 5_00:
        fvelocity.V_dalay = V_view.copy()
    if i%6_00 == 1:
        dv_dx, dv_dy = fvelocity(V_view)
    Isyn = syn(V_view)
    nodes(Isyn)

In [202]:
# dv_dx, dv_dy = fvelocity(V_view)
dv_dx_reduced, dv_dy_reduced = fvelocity.reduce_density(dv_dx, dv_dy, v_x_size=40, v_y_size=40)

In [203]:
dv_dx1, dv_dy1 = dv_dx, dv_dy
# dv_dx1, dv_dy1 = dv_dx_reduced, dv_dy_reduced

In [204]:
Ny, Nx = dv_dx1.shape  # 注意形状是 (行, 列) = (y, x)
x = np.arange(Nx)
y = np.arange(Ny)
X, Y = np.meshgrid(x, y)
step = 1
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.quiver(X[::step, ::step], Y[::step, ::step],
               -dv_dy1[::step, ::step], -dv_dx1[::step, ::step], color="b", scale=50) # , scale=500
ax2.imshow(V_view, cmap='jet', origin="lower", aspect="auto")
x_min, x_max = 35, 65
y_min, y_max = 35, 65
ax1.set_xlim(x_min, x_max)
ax1.set_ylim(y_min, y_max)
ax2.set_xlim(x_min, x_max)
ax2.set_ylim(y_min, y_max)
plt.show()

In [205]:
# spiral_wave.save_image(filename="animation.gif", duration=50)